# Setup

In [13]:
%pylab inline

import subprocess
import astropy.table
import astropy.io.fits as fits
import numpy as np
from copy import deepcopy
import random
from astropy.table import Table
import os 
import matplotlib.pyplot as plt 
import fitsio 

#directories that would be using 
locations = dict(
WLD = '/Users/Ismael/code/lensing/WeakLensingDeblending/',
WLFF = '/Users/Ismael/code/lensing/WLFF/',
aegis = '/Users/Ismael/aegis/data/',
SEx = '/Users/Ismael/aegis/data/sextractor_runs/',
aegis_slac = '/nfs/slac/g/ki/ki19/deuce/AEGIS/ismael/data',
temp_data = '/Users/Ismael/temp_data'
)
os.chdir(locations['WLD']) #set this to interested directory 

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [14]:
#files 
files = dict(
point_fits = '/Users/Ismael/temp_data/point_results.fits',
point_fits2 = '/Users/Ismael/temp_data/point_results2.fits',
point_fits3 = '/Users/Ismael/temp_data/point_results3.fits',
point_fits4 = '/Users/Ismael/temp_data/point_results4.fits',
extended_fits = '/Users/Ismael/temp_data/extended_results.fits',
)

files_temps = dict() 
for f in files: 
    l = files[f].split("/")
    temp_file = '{0}{1}'.format('/Users/Ismael/temp_data/',l[-1])
    files_temps[f] = temp_file

### Robust statistics

In [15]:
def mad(arr):
    """ Median Absolute Deviation: a "Robust" version of standard deviation.
        Indices variabililty of the sample.
        https://en.wikipedia.org/wiki/Median_absolute_deviation 
    """
    arr = np.ma.array(arr).compressed() # should be faster to not use masked arrays.
    med = np.median(arr)
    return np.median(np.abs(arr - med))

### Samples

In [16]:
#some interesting subsets of the simulation
iso_gal = lambda cat: cat[cat['purity'] > .98] #isolated galaxies
grp_gal = lambda cat: cat[cat['purity'] <= .98] #galaxies in a group of 2 or more. 

#'good' galaxies satisfy the reasonable criteria below.
good = lambda cat: cat[(cat['snr_grpf'] > 6) & (cat['sigma_m'] > .2)]

#gold sample galaxies 
gold = lambda cat: cat[(cat['ab_mag'] < 25.3)] 

#ambiguity of blends. 
ambig = lambda cat: cat[cat['ambig_blend'] == True ]
not_ambig = lambda cat: cat[cat['ambig_blend'] == False ]
detected = lambda cat: cat[cat['match'] != -1]
not_detected = lambda cat: cat[cat['match'] == -1]

#cuts 
cut_biasiso = lambda cat,bias_cut: cat[(abs(cat['bias_g1']) < bias_cut) & (abs(cat['bias_g2']) < bias_cut)]
cut_biasgrp = lambda cat,bias_cut: cat[(abs(cat['bias_g1_grp']) < bias_cut) & (abs(cat['bias_g2_grp']) < bias_cut)]
down_cut = lambda cat,param,cut: cat[cat[param] < cut]
up_cut = lambda cat,param,cut: cat[cat[param] > cut]
abs_cut = lambda cat,param,cut: cat[abs(cat[param]) < cut]
unphysical_iso = lambda cat: cat[(abs(cat['bias_g1']) > 1.) | (abs(cat['bias_g2']) > 1.)]
unphysical_grp = lambda cat: cat[(abs(cat['bias_g1_grp']) > 1.) | (abs(cat['bias_g2_grp']) > 1.)]

#more specific 
detc_and_notambig = lambda cat: cat[(cat['ambig_blend'] == False) & (cat['match'] != -1)]
notdetc_and_notambig = lambda cat: cat[(cat['ambig_blend'] == False) & (cat['match'] == -1)]
detc_and_ambig = lambda cat: cat[(cat['ambig_blend'] == True) & (cat['match'] != -1)]
notdetc_and_ambig = lambda cat: cat[(cat['ambig_blend'] == True) & (cat['match'] == -1)]
best = detc_and_notambig

In [17]:
#smaller random samples 
def leaveRandom(sz,cat):
    new_cat = deepcopy(cat)
    rm_rows = random.sample(xrange(0,len(cat)-1), len(cat) - sz)
    new_cat.remove_rows(rm_rows)
    return new_cat 

#get a smaller sample of each of the catalogs for illustrative purposes 
small = lambda cat,N: leaveRandom(N, cat)

### Read files. 

In [18]:
cats = []
names =[]
for f in files:
    cats.append(astropy.table.Table.read(fits.open(files[f]),hdu=1))
    names.append(f)

In [19]:
names

['point_fits2', 'point_fits3', 'point_fits4', 'point_fits', 'extended_fits']

# Analysis Results 

### Percetage of samples from total 

In [20]:
for i,cat in enumerate(cats): 
    print 
    print names[i] + ' catalog'
    print 'Fraction of detected & not ambiguous objects: {0:.3g}%'.format(float(len(detc_and_notambig(cat)))/len(cat)*100)
    print 'Fraction of detected & ambiguous objects: {0:.3g}%'.format(float(len(detc_and_ambig(cat)))/len(cat)*100)
    print 'Fraction of not detected & ambiguous objects: {0:.3g}%'.format(float(len(notdetc_and_ambig(cat)))/len(cat)*100)
    print 'Fraction of not detected & not ambiguous objects: {0:.3g}%'.format(float(len(notdetc_and_notambig(cat)))/len(cat)*100)
    print
    print 'Fraction of golden detected & not ambiguous objects: {0:.3g}%'.format(float(len(detc_and_notambig(gold(cat))))/len(gold(cat))*100)
    print 'Fraction of golden detected & ambiguous objects: {0:.3g}%'.format(float(len(detc_and_ambig(gold(cat))))/len(gold(cat))*100)
    print 'Fraction of golden not detected & ambiguous objects: {0:.3g}%'.format(float(len(notdetc_and_ambig(gold(cat))))/len(gold(cat))*100)
    print 'Fraction of golden not detected & not ambiguous objects: {0:.3g}%'.format(float(len(notdetc_and_notambig(gold(cat))))/len(gold(cat))*100)
    print
    print 'Fraction of good detected & not ambiguous objects: {0:.3g}%'.format(float(len(detc_and_notambig(good(cat))))/len(good(cat))*100)
    print 'Fraction of good detected & ambiguous objects: {0:.3g}%'.format(float(len(detc_and_ambig(good(cat))))/len(good(cat))*100)
    print 'Fraction of good not detected & ambiguous objects: {0:.3g}%'.format(float(len(notdetc_and_ambig(good(cat))))/len(good(cat))*100)
    print 'Fraction of good not detected & not ambiguous objects: {0:.3g}%'.format(float(len(notdetc_and_notambig(good(cat))))/len(good(cat))*100)




point_fits2 catalog
Fraction of detected & not ambiguous objects: 10.7%
Fraction of detected & ambiguous objects: 3.13%
Fraction of not detected & ambiguous objects: 3.64%
Fraction of not detected & not ambiguous objects: 82.5%

Fraction of golden detected & not ambiguous objects: 46.4%
Fraction of golden detected & ambiguous objects: 13.3%
Fraction of golden not detected & ambiguous objects: 4.22%
Fraction of golden not detected & not ambiguous objects: 36.1%

Fraction of good detected & not ambiguous objects: 44.8%
Fraction of good detected & ambiguous objects: 10%
Fraction of good not detected & ambiguous objects: 1.38%
Fraction of good not detected & not ambiguous objects: 43.9%

point_fits3 catalog
Fraction of detected & not ambiguous objects: 7.16%
Fraction of detected & ambiguous objects: 2.52%
Fraction of not detected & ambiguous objects: 3.01%
Fraction of not detected & not ambiguous objects: 87.3%

Fraction of golden detected & not ambiguous objects: 31.1%
Fraction of golden